<a href="https://colab.research.google.com/github/bathmonster/Twilio_Assesment/blob/master/Week_6_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The data set I havev choosen for this assessment is WiFi tracking and analytics. I have a Fortinet network with an enterprise firewall and Wifi. I am curiuos to what my home network can see.

I logged into my FortiAnyayler to obtain the logfiles releated to wifi traffic

the format is absoulty terrible and i had to do some manipulation of the csv file before I was comfortable using it as a dataframe

In [226]:
import csv

def remove_all_quotes_and_substrings(row):
    new_row = []
    for item in row:
        equal_index = item.find('=')
        if equal_index != -1:
            item = item[equal_index + 1:]
        item = item.replace('=', '').replace('"', '')
        new_row.append(item)
    return new_row

def process_csv(input_file, output_file):
    with open(input_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        rows = [remove_all_quotes_and_substrings(row) for row in reader]

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(rows)

if __name__ == "__main__":
    input_file = "Forti_WiFi.csv"
    output_file = "Forti_WiFi_processed.csv"

    with open(input_file, 'r') as file:
        first_line = file.readline()
        has_header = csv.Sniffer().has_header(first_line)

    if has_header:
        # Skip the header when processing
        with open(input_file, newline='') as csvfile:
            reader = csv.reader(csvfile)
            header = next(reader)
            rows = [header] + [remove_all_quotes_and_substrings(row) for row in reader]

        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(rows)
    else:
        process_csv(input_file, output_file)

    print(f"Processing completed. The result is saved in {output_file}.")


Processing completed. The result is saved in Forti_WiFi_processed.csv.


In [ ]:
import pandas as pd

# Load the Forti_WiFi dataset from the processed CSV file
# The dataset should be in the same directory as this script
Forti_WiFi_Dataset = pd.read_csv('./Forti_WiFi_processed.csv', delimiter=',', low_memory=False)

# Print the total number of WiFi log files in the dataset
print("Total WiFi Log files: " + str(len(Forti_WiFi_Dataset)))

# Display a sample of 10 random log files from the modified DataFrame
Forti_WiFi_Dataset.sample(10)


In [ ]:
import pandas as pd

# Load the Wigle WiFi dataset from the CSV file
# Ensure that the 'WigleWifi.csv' file is in the same directory as this script
Wigle_WiFi_Dataset = pd.read_csv('./WigleWifi.csv', delimiter=',', low_memory=False)

# Print the total number of WiFi log files in the dataset
print("Total WiFi Log files: " + str(len(Wigle_WiFi_Dataset)))


# Display a sample of 10 random log files from the modified DataFrame
Wigle_WiFi_Dataset.sample(10)


In [208]:
import pandas as pd

# Assuming you have Forti_WiFi_Dataframe and Wigle_WiFi_Dataset defined

# Merge the dataframes based on matching SSIDs
radiosignal_df = pd.merge(Wigle_WiFi_Dataset, Forti_WiFi_Dataset, on='ssid', how='left')

# Drop duplicates based on the 'mac' column
radiosignal_df = radiosignal_df.drop_duplicates(subset=['mac'])


# Columns to remove
columns_to_remove = [
    'FirstSeen', 'AltitudeMeters', 'AccuracyMeters',
    'itime', 'vd', 'age', 'age', 'ap', 'apscan', 'apstatus',
    'aptype', 'authserver', 'bid', 'detectionmethod', 'onwire',
    'radioid', 'radioidclosest', 'radioiddetected', 'rate',
    'snclosest', 'snr', 'srcip', 'stamac', 'vap'
]
radiosignal_df = radiosignal_df.drop(columns=columns_to_remove)

# Optional: Save the final cleaned dataframe to a new file
radiosignal_df.to_csv('final_cleaned_file.csv', index=False)





What events that my Forti has seen

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Count the occurrences of each action in the 'action' column of the DataFrame
action_counts = Forti_WiFi['action'].value_counts()

# Sort the actions in descending order of combined mission counts
sorted_actions = action_counts.sort_values(ascending=False)

# Create the horizontal bar plot
plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
plt.barh(sorted_actions.index, sorted_actions.values)

# Add labels and title
plt.xlabel('Total Number of Actions')
plt.ylabel('Action')
plt.title('Total Number of Actions for Each SSID')

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming wigle_dataset and Forti_WiFi are your DataFrames

# Calculate occurrences of each SSID in wigle_dataset and Forti_WiFi
wigle_ssid_counts = wigle_dataset['ssid'].value_counts()
Forti_WiFi_ssid_counts = Forti_WiFi['ssid'].value_counts()

# Find common SSIDs
common_ssids = list(set(wigle_ssid_counts.index) & set(Forti_WiFi_ssid_counts.index))

# Filter occurrence counts for common SSIDs
wigle_common_ssid_counts = wigle_ssid_counts[common_ssids]
Forti_WiFi_common_ssid_counts = Forti_WiFi_ssid_counts[common_ssids]

# Merge the filtered occurrence counts into a single DataFrame
merged_common_counts = pd.concat([wigle_common_ssid_counts, Forti_WiFi_common_ssid_counts], axis=1, keys=['Wigle_Dataset', 'Forti_WiFi_Dataset'])
merged_common_counts = merged_common_counts.fillna(0)  # Fill NaN values with 0

# Create a bar plot
plt.figure(figsize=(12, 6))
merged_common_counts.plot(kind='bar')
plt.title('Occurrences of Common SSIDs in wigle_dataset and Forti_WiFi')
plt.xlabel('SSID')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Display the plot
plt.show()


the occurances of AP's found in the wigle dataset that the Forti dataset found

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming wigle_dataset and Forti_WiFi are your DataFrames

# Calculate occurrences of each SSID in wigle_dataset and Forti_WiFi
wigle_ssid_counts = wigle_dataset['ssid'].value_counts()
Forti_WiFi_ssid_counts = Forti_WiFi['ssid'].value_counts()

# Find common SSIDs
common_ssids = list(set(wigle_ssid_counts.index) & set(Forti_WiFi_ssid_counts.index))

# Filter occurrence counts for common SSIDs in Forti_WiFi
Forti_WiFi_common_ssid_counts = Forti_WiFi_ssid_counts[common_ssids]

# Create a bar plot
plt.figure(figsize=(12, 6))
Forti_WiFi_common_ssid_counts.plot(kind='bar')
plt.title('Occurrences of Common SSIDs in Forti_WiFi Dataset')
plt.xlabel('SSID')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Display the plot
plt.show()


Number of time the wigle data set found my home ssid's

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming wigle_dataset and Forti_WiFi are your DataFrames

# Calculate occurrences of each SSID in wigle_dataset and Forti_WiFi
wigle_ssid_counts = wigle_dataset['ssid'].value_counts()
Forti_WiFi_ssid_counts = Forti_WiFi['ssid'].value_counts()

# Find common SSIDs
common_ssids = list (set(wigle_ssid_counts.index) & set(Forti_WiFi_ssid_counts.index))

# Filter occurrence counts for common SSIDs in wigle_dataset
wigle_common_ssid_counts = wigle_ssid_counts[common_ssids]

# Create a bar plot
plt.figure(figsize=(12, 6))
wigle_common_ssid_counts.plot(kind='bar')
plt.title('Occurrences of Common SSIDs in wigle_dataset')
plt.xlabel('SSID')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Display the plot
plt.show()



In [ ]:
import folium
from folium.plugins import HeatMap
from IPython.display import display

def main():
    """
    Create a map, add markers and a heat map layer using radiosignal_df, and display the map.
    """

    # Assuming radiosignal_df is the DataFrame
    # Drop rows with missing latitude or longitude values
    data = radiosignal_df.dropna(subset=['CurrentLatitude', 'CurrentLongitude'])

    # Set the map center to the specified latitude and longitude
    map_center = [-27.80196737, 152.9346552]
    mymap = folium.Map(location=map_center, zoom_start=16)

    # Create a list to hold heat map data points
    heat_data = []
    found_rows = []
    # Iterate through the dataset and add data points to the map
    for index, row in data.iterrows():
        lat = row['CurrentLatitude']
        lon = row['CurrentLongitude']
        ssid = row['ssid']

        # Create a marker with icon for each SSID
        marker = folium.Marker([lat, lon], popup=ssid, icon=folium.Icon(icon='cloud'))
        marker.add_to(mymap)

        # Add the location to the heat data
        heat_data.append([lat, lon])

        # Add relevant row data to the list
        found_rows.append({'CurrentLatitude': lat, 'CurrentLongitude': lon, 'ssid': ssid})

    # Create a HeatMap layer using the heat data
    HeatMap(heat_data, radius=20).add_to(mymap)

    # Display the map on the screen
    display(mymap)


if __name__ == "__main__":
    main()


Code to show what my fortirouter sees that matchs what the wiggle data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming merged_wifi is your merged dataset

# Calculate the occurrences of each SSID in the merged dataset
ssid_counts = merged_data['ssid'].value_counts()

# Create a bar plot
plt.figure(figsize=(10, 6))
ssid_counts.plot(kind='bar')
plt.title('Occurrences of SSIDs in Merged WiFi Dataset')
plt.xlabel('SSID')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Specify the actions to filter
specified_actions = [
    'client-deauthentication',
    'client-retry',
    'client-association-failure',
    'client-disconnected-by-wtp',
    'rogue-ap-off-air',
    'rogue-ap-changed'
]

# Filter the DataFrame to include only the specified actions
filtered_data = Forti_WiFi[Forti_WiFi['action'].isin(specified_actions)]

# Count the occurrences of each action in the filtered DataFrame
action_counts = filtered_data['action'].value_counts()

# Sort the actions in descending order of counts
sorted_actions = action_counts.sort_values(ascending=False)

# Create the horizontal bar plot
plt.figure(figsize=(10, 6))
plt.barh(sorted_actions.index, sorted_actions.values)

# Add labels and title
plt.xlabel('Total Number of Actions')
plt.ylabel('Action')
plt.title('Total Number of Specified Actions')

# Show the plot
plt.show()


In [ ]:
import folium
from folium.plugins import HeatMap
from IPython.display import display

def main():
    """
    Create a map, add markers and a heat map layer using radiosignal_df, and display the map.
    """

    # Assuming radiosignal_df is the DataFrame
    # Drop rows with missing latitude or longitude values
    data = radiosignal_df.dropna(subset=['CurrentLatitude', 'CurrentLongitude'])

    # Set the map center to the specified latitude and longitude
    map_center = [-27.80196737, 152.9346552]
    mymap = folium.Map(location=map_center, zoom_start=16)

    # Create a list to hold heat map data points
    heat_data = []
    found_rows = []
    # Iterate through the dataset and add data points to the map
    for index, row in data.iterrows():
        lat = row['CurrentLatitude']
        lon = row['CurrentLongitude']
        ssid = row['ssid']

        # Create a marker with icon for each SSID
        marker = folium.Marker([lat, lon], popup=ssid, icon=folium.Icon(icon='cloud'))
        marker.add_to(mymap)

        # Add the location to the heat data
        heat_data.append([lat, lon])

        # Add relevant row data to the list
        found_rows.append({'CurrentLatitude': lat, 'CurrentLongitude': lon, 'ssid': ssid})

    # Create a HeatMap layer using the heat data
    HeatMap(heat_data, radius=20).add_to(mymap)

    # Display the map on the screen
    display(mymap)


if __name__ == "__main__":
    main()
